In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Aux functions

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [3]:
# convert series to supervised learning
def series_to_supervised_miso(data, n_in, n_out, endog_var, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[endog_var].shift(-i))
		if i == 0:
			names += [(endog_var+'(t)')]
		else:
			names += [(endog_var+'(t+%d)' % (i))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

## Dataset Smart Home with Weather Information

In [6]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/HomeC.csv')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
data = data.loc[0: : 10]
data
data.dropna(inplace=True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
10,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
20,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
30,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
40,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [7]:
round(data.describe(),3)

,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
count,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,...,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000,50391.000
mean,0.860,0.076,0.031,0.099,0.138,0.081,0.064,0.042,0.014,0.003,...,50.742,0.664,9.253,48.264,1016.302,6.649,202.390,0.003,38.693,0.056
std,1.058,0.129,0.190,0.168,0.180,0.104,0.077,0.058,0.012,0.024,...,19.115,0.194,1.612,22.030,7.895,3.982,106.544,0.011,19.088,0.166
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-12.640,0.130,0.270,-32.080,986.400,0.000,0.000,0.000,-27.240,0.000
25%,0.370,0.003,0.000,0.020,0.065,0.040,0.005,0.007,0.013,0.000,...,35.770,0.510,9.420,31.090,1011.290,3.660,148.000,0.000,24.600,0.000
50%,0.564,0.004,0.000,0.021,0.067,0.042,0.005,0.008,0.013,0.001,...,50.320,0.680,10.000,50.320,1016.530,5.930,208.000,0.000,39.030,0.000
75%,0.974,0.084,0.000,0.069,0.081,0.069,0.125,0.052,0.013,0.001,...,66.255,0.840,10.000,66.255,1021.480,8.940,295.000,0.000,54.790,0.000
max,12.819,0.591,1.400,1.880,0.795,0.734,0.786,1.229,1.014,0.916,...,93.720,0.980,10.000,101.120,1042.460,22.910,359.000,0.191,75.490,0.840


### Dataset Energy Appliance: Preprocessing, Lag = 1, t+1

In [8]:
data = series_to_supervised_miso(data, 1, 1, endog_var='use [kW]', dropnan=True)
data.head()

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,humidity(t-1),visibility(t-1),apparentTemperature(t-1),pressure(t-1),windSpeed(t-1),windBearing(t-1),precipIntensity(t-1),dewPoint(t-1),precipProbability(t-1),use [kW](t)
10,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.585083
20,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.533817
30,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.190950
40,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.582083
50,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.837000


In [9]:
# data = clean_dataset(data)


## Model: PcaWeightedMVFTS - PCA-MVFTS

In [10]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
                       endogen_variable = 'use [kW](t-1)',
                       n_part = 50)

In [13]:
def sliding_window_pca(data,n_windows,train_size,dim,fs):

    result = {
         "dim":[],
         "fs":[],
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": [],
         "mae": [],
         "nrmse":[],
         "rules":[]
    }

    
    order = 1
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:

            pca_wmvfts = PcaWeightedMVFTS(n_components = dim,
                       endogen_variable = 'use [kW](t-1)',
                       n_part = fs)
            
            data_train = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            data_test = ttest.loc[:,'gen [kW](t-1)':'use [kW](t)']
            
             #ALtera o nome da columa porque o modelo utiliza esse label como variável endogena 
            data_test.rename(columns = {'use [kW](t)': 'use [kW](t-1)'}, inplace = True)
            
            model, pca_reduced_train = pca_wmvfts.run_train_model(data_train)
            forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,data_test)
            
            #Teste e cálculo dos erros da previsão
            #valor use [kW](t-1) é o use [kW](t) --> label igual por causa do modelo  
            rmse = Measures.rmse(pca_reduced_test['use [kW](t-1)'],forecast)
            mape = Measures.mape(pca_reduced_test['use [kW](t-1)'],forecast)
            smape = Measures.smape(pca_reduced_test['use [kW](t-1)'],forecast)
            
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(pca_reduced_test['use [kW](t-1)'], forecast)

            nrmse = cal_nrmse(rmse, pca_reduced_test['use [kW](t-1)'])

            result["dim"].append(dim)
            result["fs"].append(fs)
            
            result["rmse"].append(round(rmse,3))
            result["mape"].append(round(mape,3))
            result["smape"].append(round(smape,3))
            result["mae"].append(round(mae,3))
            result["nrmse"].append(round(nrmse,3))
            result["rules"].append(len(model))
            result["window"].append(ct)
        
    measures = pd.DataFrame(result)
    return measures



In [14]:
dimensions = [2,3,4,5]
fuzzy_sets = [10,20,30,40,50]

final_result = {
    "dimensions": [],
    "fuzzy_sets": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "smape": [],
    "nrmse":[],
    "rules":[],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "smape_std": [],
    "nrmse_std": [],
    "rules_std":[],
}

n_windows = 30
train_size = 0.75

df_result_win = pd.DataFrame(columns=list(['dim','fs','window','rmse','mape','smape','mae','nrmse','rules']))

for dim in dimensions:
    for fs in fuzzy_sets:
        measures = sliding_window_pca(data,n_windows,train_size,dim,fs)
        final_result["dimensions"].append(dim)
        final_result["fuzzy_sets"].append(fs)
        
        rmse = round(statistics.mean(measures['rmse']),3)
        mape = round(statistics.mean(measures['mape']),3)
        smape = round(statistics.mean(measures['smape']),3)
        mae = round(statistics.mean(measures['mae']),3)
        nrmse = round(statistics.mean(measures['nrmse']),3)
        rules = round(statistics.mean(measures['rules']),3)
        
        rmse_std = round(statistics.stdev(measures['rmse']),3)
        mape_std = round(statistics.stdev(measures['mape']),3)
        smape_std = round(statistics.stdev(measures['smape']),3)
        mae_std = round(statistics.stdev(measures['mae']),3)
        nrmse_std = round(statistics.stdev(measures['nrmse']),3)
        rules_std = round(statistics.stdev(measures['rules']),3)
        
        final_result["rmse"].append(rmse)
        final_result["mape"].append(mape)
        final_result["smape"].append(smape)
        final_result["mae"].append(mae)
        final_result["nrmse"].append(nrmse)
        final_result["rules"].append(rules)
        
        final_result["rmse_std"].append(rmse_std)
        final_result["mape_std"].append(mape_std)
        final_result["smape_std"].append(smape_std)
        final_result["mae_std"].append(mae_std)
        final_result["nrmse_std"].append(nrmse_std)
        final_result["rules_std"].append(rules_std)
        
        print(f'Results: {(dim,fs,rmse,mae,mape,smape,nrmse,rules)}')
        
        df_result_win = pd.concat([df_result_win,measures],axis=0) 
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics PCA-WMVFTS (test): ")
final_measures_pca

/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/hom

/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/hom

Results: (2, 10, 0.424, 0.306, 177.341, 18.895, 0.081, 427.933)


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/hom

/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/hom

...

Results: (5, 40, 0.001, 0.002, 0.208, 0.055, 0.0, 55388.833)


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/hom

/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/hom

Results: (5, 50, 0.0, 0.002, 0.197, 0.049, 0.0, 59766.567)
Statistics PCA-WMVFTS (test): 


,dimensions,fuzzy_sets,rmse,mae,mape,smape,nrmse,rules,rmse_std,mae_std,mape_std,smape_std,nrmse_std,rules_std
0,2,10,0.424,0.306,177.341,18.895,0.081,427.933,0.390,0.295,345.486,8.299,0.035,70.018
1,2,20,0.310,0.188,83.209,12.102,0.058,1420.433,0.310,0.194,185.737,6.629,0.028,259.286
2,2,30,0.266,0.131,49.896,8.401,0.050,2531.400,0.259,0.127,109.441,5.119,0.028,464.435
3,2,40,0.193,0.087,33.955,5.882,0.038,3527.600,0.168,0.078,86.522,3.604,0.022,633.657
4,2,50,0.169,0.062,17.823,4.285,0.034,4392.133,0.135,0.047,28.321,2.479,0.019,743.598
5,3,10,0.275,0.186,97.914,12.958,0.054,1891.833,0.238,0.172,217.493,6.545,0.023,289.699
6,3,20,0.143,0.056,15.838,4.171,0.028,5552.167,0.124,0.043,20.344,2.307,0.016,883.163
7,3,30,0.102,0.024,5.619,1.678,0.019,8545.833,0.126,0.020,5.891,1.118,0.014,1364.805
8,3,40,0.055,0.012,2.548,0.845,0.011,10652.167,0.048,0.010,2.219,0.599,0.010,1660.391
9,3,50,0.040,0.008,1.450,0.482,0.008,12207.933,0.064,0.011,1.587,0.413,0.008,1772.687


In [16]:
round(final_measures_pca,4)

,dimensions,fuzzy_sets,rmse,mae,mape,smape,nrmse,rules,rmse_std,mae_std,mape_std,smape_std,nrmse_std,rules_std
0,2,10,0.424,0.306,177.341,18.895,0.081,427.933,0.390,0.295,345.486,8.299,0.035,70.018
1,2,20,0.310,0.188,83.209,12.102,0.058,1420.433,0.310,0.194,185.737,6.629,0.028,259.286
2,2,30,0.266,0.131,49.896,8.401,0.050,2531.400,0.259,0.127,109.441,5.119,0.028,464.435
3,2,40,0.193,0.087,33.955,5.882,0.038,3527.600,0.168,0.078,86.522,3.604,0.022,633.657
4,2,50,0.169,0.062,17.823,4.285,0.034,4392.133,0.135,0.047,28.321,2.479,0.019,743.598
5,3,10,0.275,0.186,97.914,12.958,0.054,1891.833,0.238,0.172,217.493,6.545,0.023,289.699
6,3,20,0.143,0.056,15.838,4.171,0.028,5552.167,0.124,0.043,20.344,2.307,0.016,883.163
7,3,30,0.102,0.024,5.619,1.678,0.019,8545.833,0.126,0.020,5.891,1.118,0.014,1364.805
8,3,40,0.055,0.012,2.548,0.845,0.011,10652.167,0.048,0.010,2.219,0.599,0.010,1660.391
9,3,50,0.040,0.008,1.450,0.482,0.008,12207.933,0.064,0.011,1.587,0.413,0.008,1772.687


In [22]:
final_measures_pca.to_csv (r'pca_wmvfts_dim_fsets_ksh.csv', index = False, header=True)

In [20]:
df_result_win.to_csv (r'wim_pca_wmvfts_dim_fsets_ksh.csv', index = False, header=True)

In [21]:
df_result_win

,dim,fs,window,rmse,mape,smape,mae,nrmse,rules
0,2,10,0,0.322,31.110,13.173,0.254,0.077,519
1,2,10,1679,0.250,17.482,8.073,0.167,0.061,495
2,2,10,3358,0.251,40.017,14.195,0.188,0.089,498
3,2,10,5037,0.396,22.268,10.937,0.280,0.079,421
4,2,10,6716,0.368,125.558,20.168,0.282,0.101,455
...,...,...,...,...,...,...,...,...,...
25,5,50,41975,0.000,0.029,0.014,0.000,0.000,57122
26,5,50,43654,0.000,0.036,0.018,0.000,0.000,53234
27,5,50,45333,0.002,0.064,0.032,0.000,0.001,55721
28,5,50,47012,0.000,1.483,0.192,0.000,0.000,59028
